In [30]:
import pandas as pd
import os
import asyncio
import aiohttp
import time

In [31]:
async def fetch_async(f):
    print("Started", f.split("/").pop())
    with open(f) as cluster:
        params = {
        "ontology": 'panther_bp',
        "input": "".join((protein for protein in cluster)),
        "species": "HUMAN",
        "correction": "bonferroni",
        "format": "plain"
        }   
    await asyncio.sleep(0)
    r = await aiohttp.request('GET', "http://pantherdb.org/webservices/go/overrep.jsp?", params=params)
    j = await r.json()
    r.close()
    sig = await sigTest(f, j)
    return sig

In [32]:
async def asynchronous():
    d = "/Users/harry/git/fly/clustering/clusters_t0/"
    jobs = [os.path.join(d, f) for f in os.listdir(d)]
    futures = [fetch_async(i) for i in jobs]
    for i, future in enumerate(asyncio.as_completed(futures)):
        result = await future
        print(result)

In [33]:
async def sigTest(f, r):
    sig = (i for i in r['results']['result'] if i['pValue'] < 0.05)
    df = pd.io.json.json_normalize(list(sig)).sort_values('pValue')
    return df

In [34]:
asyncio.set_event_loop(asyncio.new_event_loop())
ioloop = asyncio.get_event_loop()
ioloop.run_until_complete(asynchronous())
ioloop.close()

Started clusters_WT_3.csv
Started clusters_WT_7.csv
Started clusters_AB_5.csv
Started clusters_AB_2.csv
Started clusters_WT_6.csv
Started clusters_AB_1.csv
Started clusters_AB_3.csv
Started clusters_WT_1.csv
Started clusters_WT_4.csv
Started clusters_AB_4.csv
Started clusters_WT_5.csv
Started clusters_AB_7.csv
Started clusters_AB_6.csv
Started clusters_AB_8.csv
Started clusters_AB_9.csv
Started clusters_WT_2.csv
    expected  number_in_list  number_in_reference    pValue plus_minus  \
0  25.521934              19                 8633  0.000000          -   
1   0.100515               4                   34  0.000861          +   
2   2.332539              10                  789  0.024463          +   

      term.id                 term.label  
0         NaN               UNCLASSIFIED  
1  GO:0006635  fatty acid beta-oxidation  
2  GO:0009056          catabolic process  
     expected  number_in_list  number_in_reference    pValue plus_minus  \
0  137.489128             105           